# Cleaning Up the code

## Comms Check

**Here is the first check we coded, which checked comms in the loads. Notice that comments have been added and uneeded code has been removed for clarity and responsiveness.**

In [3]:
from re import search
from glob import glob

# This function will take a Backstop file 
# and find all the comms that are contained
# A count is returned to the base program

def countComms(backstop_filename):
    # Open the file for reading
    file_in = open(backstop_filename,'r')
    
    # Counter for the amount of comms in the file
    counter=0

    for line in file_in:
        # regular expression search of the current line 
        # from the file
        ctx = search('TLMSID=.(CTX[A,B]ON)',line)
        if ctx:
            # add to the count
            counter+=1
    # Close that file, private pyle!
    file_in.close()
    
    # return your answer out of the  function to 
    # allow the main program to use it
    return counter

**Let's try the function out and see how it works!**

In [5]:
import sys

# Find all backstop files in the Backstop directory
file = glob("/home/mission/Backstop/MAR1918B/*backstop")

# Check if we have only one. Otherwise, slap their hand, and exit
if not(len(file) > 0 and len(file) <2):
    print("Incorrect Amount of Backstop files. Exiting...")
    sys.exit()
else:
    # Do the things
    backstop_file = file[0]

    counter = countComms(backstop_file)

    if counter < 1:
        print("There are no comms included in this loadset.")
    else:
        print("There are " + str(counter) + " comms in this loadset.")

There are 19 comms in this loadset.


**This seems to work great! Let's move on to the next one.**

## Bias Check

** Once again, the code has been cleaned up and beautfied. The comments were left out for us to do.**

In [47]:
from re import search
from glob import glob
from Chandra.Time import DateTime

def checkBiasVsFormat(backstop_filename):
    regex_to_match = ["(\d+:\d+:\d+:\d+:\d+.\d+).*TLMSID= (X[T C]Z[0-9 A-F]+)",
                         "(\d+:\d+:\d+:\d+:\d+.\d+).*TLMSID= (CSELFMT\d)"] 

    file_in = open(backstop_filename)
    results_array = []

    for line in file_in:
        match = search(regex_to_match[0],line)
        if not match:
            match = search(regex_to_match[1],line)
        if match:
            temp_object = {
                "timestring": match.group(1),
                "tlmsid":match.group(2),
                "time_seconds":DateTime(match.group(1)).secs
            }

            results_array.append(temp_object)
    file_in.close()
    
    previous_object = {}
    flag=0
    for index,result_instance in enumerate(results_array):
        if not index == 0:
            time = result_instance['time_seconds']
            previous_time = previous_object['time_seconds']

            match=search("SELFMT\d",result_instance['tlmsid'])
            if match:
                match2=search("SELFMT\d",previous_object['tlmsid'])

                if not match2:
                    if  previous_time+(26*60) > time:
                        flag=1
                        output = previous_object['tlmsid']+" "+previous_object['timestring'] +\
                        " | "+ result_instance['tlmsid'] +" " +\
                        result_instance['timestring'] + "|<---THIS NEEDS CHECKED | TIME BETWEEN: "+\
                        str(round((result_instance['time_seconds']-previous_object['time_seconds'])/60,2))+' Minutes'

                        print(output)

        previous_object = result_instance
        
    if not flag:
        print('There is no format commanding during an ACIS bias.')
        



**Since we already have a Backstop to try this on, let's use it!!!**

In [48]:
checkBiasVsFormat(backstop_file)

There is no format commanding during an ACIS bias.
